# Power BI



# 🔵 **1. LOYALLIQ ÜZRƏ DATA ANALİTİK NƏ İŞ GÖRÜR?**

Bu analitik **müştəri davranışını araşdırır**, məqsəd — müştərilərin şirkətdə qalmasını artırmaqdır (retention + loyalty).

### 🔹 Əsas işləri:

1. **Müştəri davranışı analizi**

   * Müştəri nə alır?
   * Nə qədər tez-tez alır?
   * Hansı məhsulu daha çox sevir?

2. **Churn analysisi** (müştərinin şirkətdən ayrılıb-ayrılmayacağını proqnozlaşdırmaq)

   * XGBoost, Random Forest, Logistic Regression
   * Churn score yaratmaq

3. **Segmentasiya (K-means, clustering)**

   * VIP müştəri kimdir?
   * Endirimə həssas müştəri kimdir?
   * Sadə müştəri kimdir?

4. **Loyallıq proqramlarının qiymətləndirilməsi**

   * Bonus proqramı işləyir?
   * Kampaniya effektlidir?
   * Endirim xərci geri qayıdır? (ROI)

5. **Customer Lifetime Value (CLV)** hesablanması

   * Bir müştəri şirkətə uzun müddətdə nə qazandırır?

6. **Kampaniya A/B testləri**

   * SMS kampaniyası göndərdim → dönüş (conversion) neçə faizdir?
   * Hansı təklif daha yaxşı işləyir?

7. **Dashboardlar & Reportlar**

   * Power BI / Tableau ilə loyallıq göstəriciləri (retention, repeat rate, churn rate).

---

# 🔴 **2. FRAUD ÜZRƏ DATA ANALİTİK NƏ İŞ GÖRÜR?**

Fraud (dələduzluq) analitiki **fırıldaq əməliyyatları aşkar etmək**, gecikmədən xəbərdarlıq vermək, sistemin təhlükəsizliyini qorumaq üçün data analizi aparır.

### 🔹 Əsas işləri:

1. **Şübhəli tranzaksiyaların analizi**

   * Kart oğurluğu
   * E-commerce fırıldaqları
   * Fake account
   * Şübhəli pul köçürmələri

2. **Fraud detection modelləri**

   * Anomaly detection (Isolation Forest, LOF)
   * Classification (XGBoost, CatBoost, RF)
   * Rule-based sistemlər

3. **Real-time monitoring**

   * Şübhəli əməliyyatı dərhal bloklamaq üçün alert sistemləri qurur.

4. **Feature engineering**

   * Hər müştərinin tipik davranış patterni çıxarılır
   * Sonra həmin pattern pozulanda fraud siqnalı yaranır

5. **Risk scoring sistemi**

   * Hər əməliyyata “risk balı” verilir
   * Məsələn: 0–100 arası risk score

6. **ML modellərinin optimizasiyası**

   * False positive-ləri azaltmaq
   * Modelin sürətli işləməsini təmin etmək

7. **Fraud ssenarilərinin yazılması və test olunması**

   * Məsələn:

     * 1 saniyə içində 5 tranzaksiya → şübhəli
     * Kart başqa ölkədə birdən istifadə olunursa → şübhəli

---

# 🟢 **Bu iki rolun fərqi nədir?**

| Sahə                   | Məqsəd                                 | İstifadə olunan modellər          | Fokus                 |
| ---------------------- | -------------------------------------- | --------------------------------- | --------------------- |
| **Loyallıq Analitiki** | Müştərini itirməmək, alışları artırmaq | Segmentation, Churn, CLV          | Müştəri davranışı     |
| **Fraud Analitiki**    | Fırıldaq əməliyyatlarını aşkar etmək   | Anomaly detection, Classification | Təhlükəsizlik və risk |

---

# ⚙️ Hər iki vəzifədə tələb olunan texnologiyalar

* **Python** (pandas, sklearn, numpy)
* **SQL** (çox güclü səviyyədə)
* Power BI / Tableau
* ML modelləri
* Statistical analysis

---

İstəsən, hər iki rol üçün **CV-də nələri yazmaq olar**, və ya **müsahibə suallarını** da hazırlaya bilərəm.




# 🔵 1. **Retention Rate** (Müştərinin geri qayıtma faizi)

**Məntiq:**
Bir ay alış edən müştərilər → növbəti ay yenidən alış edən müştərilər.

### 📌 DAX formulu:

```DAX
Returning Customers =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    FILTER(
        Sales,
        Sales[OrderDate] >= EDATE(MAX(Sales[OrderDate]), -1)
        && Sales[OrderDate] < MAX(Sales[OrderDate])
    )
)
```

```DAX
Retention Rate =
DIVIDE([Returning Customers], [Active Customers Previous Month])
```

---

# 🟢 2. **Repeat Rate** (Təkrar alış faizi)

**Məntiq:**
Bir müştəri ən azı 2 dəfə alış edibsə → repeat customer sayılır.

### 📌 DAX:

```DAX
Repeat Customers =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    FILTER(
        VALUES(Sales[CustomerID]),
        CALCULATE(COUNTROWS(Sales)) > 1
    )
)
```

```DAX
Repeat Rate =
DIVIDE([Repeat Customers], [Total Customers])
```

---

# 🔴 3. **Churn Rate** (İtirilmiş müştərilər)

**Məntiq:**
Keçən ay aktiv olub bu ay alış etməyən müştərilər = churn.

### 📌 DAX:

```DAX
Lost Customers =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    FILTER(
        VALUES(Sales[CustomerID]),
        NOT(
            Sales[CustomerID] IN
            CALCULATETABLE(
                VALUES(Sales[CustomerID]),
                FILTER(Sales, Sales[OrderDate] >= STARTOFMONTH(Sales[OrderDate])
                                && Sales[OrderDate] < MAX(Sales[OrderDate]))
            )
        )
    )
)
```

```DAX
Churn Rate =
DIVIDE([Lost Customers], [Active Customers Previous Month])
```

---

# 🟣 Power BI-də addım-addım etmə qaydası

### **1) Data import**

* Excel / SQL-dən **Sales** datasını gətirirsən.
  Lazım olan sütunlar:

  * `CustomerID`
  * `OrderID`
  * `OrderDate`
  * `Amount`

### **2) Calendar Table (date table) yaradılır**

Bu lazımdır ki, aylıq hesablamalar düzgün işləsin.

DAX ilə yarat:

```DAX
Calendar = CALENDAR(MIN(Sales[OrderDate]), MAX(Sales[OrderDate]))
```

Sonra əlaqə qur:

* Calendar[Date] → Sales[OrderDate] (1 to many relationship)

### **3) Metrics (Measures) yarat**

Modelling → New Measure → formulları yazırsan.

### **4) Visual-lar**

Power BI-də aşağıdakı vizualları qoy:

| Visual     | Metrik               |
| ---------- | -------------------- |
| Card       | Retention Rate       |
| Card       | Repeat Rate          |
| Card       | Churn Rate           |
| Line Chart | Time-based retention |
| Bar Chart  | Customer segments    |

---

# 🧩 İstəsən mən sənə **hazır Power BI .pbix template** hazırlayıb verə bilərəm

Tam içində:

* DAX formullar
* Retention vizual
* Churn vizual
* Repeat vizual
* Customer segmentation

Sadəcə datasının nümunəsini göndər.




## ✅ Başqa vacib metriklər və analizlər

### • Customer Lifetime Value (CLV / LTV)

* CLV — “bir müştərinin şirkətə ömrü boyunca gətirəcəyi dəyər”dir. ([Википедия][1])
* Hesablama dəstəyi üçün:

  * Ortalama alış dəyəri (Average Order Value – AOV)
  * Alışların sayı / tezliyi (Purchase Frequency)
  * Müştərinin şirkətlə qalma müddəti (Customer Lifetime) və ya Churn / Retention dərəcəsi. ([Medium][2])
* CLV-lə bilərsən hansı müştərilərin “həqiqətən dəyərli” olduğunu — bu da loyallıq strategiyalarını planlamaqda kömək edir. ([data-nimble.com][3])

### • RFM Analizi (Recency / Frequency / Monetary)

* “Recency” — son alış vaxtı
* “Frequency” — alış tezliyi
* “Monetary” — nə qədər xərcləmiş
  Bu üç metriklə müştəriləri seqmentlərə ayıra bilərsən: loyalları, “risk altında” olanları, yüksək dəyərli müştəriləri və s. ([help.patchretention.com][4])
* Belə seqmentasiya sayəsində marketinq kampaniyalarını daha effektiv hədəfləyə bilərsən. ([LinkedIn][5])

### • Cohort Analizi (kohort analiz)

* Məsələn: “2025-ci ilin yanvar ayında gələn yeni müştərilər nə qədər müddətdə geri alış edirlər?” kimi analiz. Bu, retention-in zamanla necə dəyişdiyini göstərir. ([proanalytics.team][6])
* Kohort + Retention + Revenue (gəlir) analizi edərək, həm müştəri, həm də gəlir baxımından uzun müddətlik trendi görə bilərsən. ([proanalytics.team][6])

### • Ortalama Alış Dəyəri (AOV) və Alış Tezliyi (Purchase Frequency)

* AOV = ümumi gəlir / toplam satış sayı. ([Medium][7])
* Tezlik = satış sayı / müştəri sayı (və ya müştərinin alış etdiyi dövrlər). Bu metriklər satış davranışını anlamaqda faydalıdır. ([Medium][2])

### • Gəlir saxlanma (Revenue Retention) & Segment-based gəlir

* Yalnız “müştəri saxlamaq” yox, “neçə gəlir qalır” baxımından da baxmaq vacibdir. Bəzən müştərilər qalır, amma xərcləri azalır — bu zaman gəlir retention düşə bilər. ([proanalytics.team][6])
* Müştəri seqmentləri üzrə analiz: “Hər seqment şirkətə nə qədər gəlir gətirir?”, “Hansı seqmentlər risklidir?” və s.

### • Aktivlik / Engagement metrikləri (subscription modellərində, SaaS, e-commerce və s.)

* Məsələn: abunə yenilənmə (renewal) nisbəti, aktiv hesablar, istifadə tezliyi, “aha moment”–lər. ([predictive-marketing.com][8])

---

## 🛠 Power BI-də bunları necə tətbiq etmək olar

* Mövcud satış / müştəri datası üzərində **Calendar table**, **Customer table** və **Transaction/Sales** table yaradıb düzgün model qur.
* CLV, AOV, Purchase Frequency, RFM skoru kimi **Measure**lər (DAX) yarat.
* **Segmentasiya üçün calculated column** / measure — müştəriləri RFM, CLV seqmentlərinə ayır.
* **Visuals**: bar chart, heatmap (RFM segmentləri üçün), cohort-heatmap (cohort analiz üçün), KPI card-lar (CLV, Avg Order Value), funnel chart (müştəri yolçuluğu), slicers (tarix, segment, region vs).
* Vaxtla (aylıq, kvartal, illik) analiz — trend görmək üçün line chart / area chart.

---

## 🎯 Niyə buna dəyər?

* Sadəcə “neçə müştəri getdi / neçə qaldı” yox — “müştərinin dəyəri nədir” baxışı verir (profitability over time).
* Marketinq, kampaniya, loyallıq proqramlarını daha hədəflənmiş planlaşdırmaq.
* “Yüksək dəyərli müştəri”ləri tanıyıb onlara fokuslamaq, xərcləri optimallaşdırmaq.
* Şirkət üçün strateji qərarlar verməkdə — acquisition vs retention balansı, büdcə bölgüsü, qiymət siyasəti və s.

---

Əgər istəyirsənsə — **əslində istifadə olunan 5–10 “ən faydalı KPI & metrik”dən ibarət siyahı** hazırlaya bilərəm — satış / e-commerce / subscription / bank sferalarında — və Power BI-də necə hesablamaq olar, DAX nümunələri ilə birlikdə. Həmçinin **dashboard nümunəsi strukturu** yaza bilərəm — beləcə öz layihəndə tez tətbiq edə bilərsən. İstərsən?

[1]: https://en.wikipedia.org/wiki/Customer_lifetime_value?utm_source=chatgpt.com "Customer lifetime value"
[2]: https://medium.com/%40alperennminnett/crm-rfm-ve-cltv-tahmini-8c4c6adafa5e?utm_source=chatgpt.com "CRM, RFM ve CLTV Tahmini. CRM( Customer Relationship Management ) | by Alperen Minnet | Medium"
[3]: https://data-nimble.com/sales-kpis-power-bi?utm_source=chatgpt.com "Sales KPIs with Power BI — DataNimble"
[4]: https://help.patchretention.com/dashboards-retention-overview?utm_source=chatgpt.com "Retention Dashboard: Analytics Overview"
[5]: https://www.linkedin.com/posts/thanisha-ganesh-557820281_powerbi-rfmanalysis-customersegmentation-activity-7348746657938763776-WR9-?utm_source=chatgpt.com "How to Predict Customer Churn with RFM Analysis in Power BI | Thanisha Ganesh posted on the topic | LinkedIn"
[6]: https://proanalytics.team/blog/cohort-analysis-definition-types-and-examples?utm_source=chatgpt.com "What Is Cohort Analysis: Definition, Retention, SaaS & Power BI Examples | PROANALYTICS"
[7]: https://oguzerdo.medium.com/crm-analytics-cltv-hesaplama-d75cac75a5dd?utm_source=chatgpt.com "CRM Analytics — CLTV Hesaplama | by Oğuz Erdoğan | Medium"
[8]: https://predictive-marketing.com/2025/09/08/understanding-customer-analytics-practical-steps-metrics-and-tools/?utm_source=chatgpt.com "Understanding Customer Analytics: Practical Steps, Metrics, and Tools – Predictive Marketing"




## 🔎 Əlavə vacib metriklər və analizlər

| Metrik / analiz                                                     | Niyə faydalıdır                                                                                                | Qısa izah                                                                                                                         |
| ------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------- |
| **Customer Lifetime Value (CLV)**                                   | Müştərinin şirkətə ömrü boyunca gətirəcəyi dəyəri proqnozlaşdırır — hansılarını saxlamağa dəyər, hansıları yox | Müştərinin gəlirləri – xərcləri nəzərə alaraq, “bu müştəridən nə qədər gəlir götürmək olar” sualına cavab verir. ([Википедия][1]) |
| **Average Order Value (AOV)**                                       | Hər alışın orta dəyərini göstərir — endirim, kampaniya effektini ölçmək üçün                                   | Ümumi gəliri ümumi sifariş sayına bölmək kifayətdir. ([loyoly.io][2])                                                             |
| **Satın alma tezliyi (Purchase Frequency)**                         | Müştərinin nə qədər tez-tez alış etdiyini bilir — loyal, aktiv və ya nadir müştəri analizi üçün                | Məsələn: “əməkdaşlıq dövrü ərzində orta 3–4 alış/il” vs “1 alış/il” kimi. ([WebKodlama.net][3])                                   |
| **RFM analizi (Recency, Frequency, Monetary)**                      | Müştəriləri seqmentlərə ayırmağa (VIP, orta, az-aktiv və s.) imkan verir                                       | Recency = son alış tarixi, Frequency = alış sayısı, Monetary = xərcləmə məbləği. ([Medium][4])                                    |
| **Müştəri seqmentasiyası & Profilinq**                              | Müxtəlif müştəri qruplarının xüsusiyyətlərini görmək, onlara fərqli strategiyalar tətbiq etmək                 | Məsələn: yüksək dəyərli + tez-tez alan, nadir alan + yüksək məbləğ xərcləyən, riskdə olan və s. ([WebKodlama.net][3])             |
| **Gelir bazlı göstəricilər: ARPU, MRR / Subscription modellərində** | Əgər abunə (subscription/service) biznesdirsə — aylıq / periodik gəlirləri analiz edir                         | Hər istifadəçidən orta aylıq / period gəlir. ([rapitek.com][5])                                                                   |
| **Upsell / Cross-sell / Retention kampaniyalarının ölçülməsi**      | Kampaniyaların effektivliyini, müştərilərin əlavə məhsul / xidmət almasını analiz etmək                        | Məsələn: loyallıq proqramı sonrası Repeat Rate və AOV artımı. ([loyoly.io][2])                                                    |
| **Müştəri “Sağlamlıq Skoru” (Customer Health Score)**               | Müştərinin şirkətlə bağlı long-term dəyəri, risk & imkanlarını bir skorla qiymətləndirir                       | Aktivlik, xərcləmə, interaksiya (müştəri dəstəyi, feedback), məhsul istifadə tezliyi kimi dəyişənlər daxildir. ([rapitek.com][5]) |

---

## 🧰 Power BI-də bu metrikləri necə tətbiq etmək olar

Sən əvvəllər Retention / Repeat / Churn üçün DAX misalları istəmişdin. Eyni yanaşmanı bu əlavə metriklər üçün də tətbiq edə bilərsən:

* **CLV**: Müştərinin bütün alış tarixçəsini, orta alış dəyərini və müştərini saxlama ehtimalını (retention rate) götürüb, DAX və ya müşahidəyə əsaslanan sadə model edə bilərsən.
* **RFM seqmentasiyası**: - Recency, Frequency, Monetary dəyərləri üçün **Measure**-lər düzəltsən → **Cluster / Segment** yaratmaq üçün **Calculated Column** və ya **Group by** istifadə edə bilərsən. Sonra **Bar / Pie / Table vizualları** ilə seqmentləri vizuallaşdır.
* **AOV və Purchase Frequency**: Hər sifariş / müştəri üçün orta dəyər + aylıq / dövri analiz → Power BI-də Card, KPI və Line chart.
* **Segment yönümlü analiz + kampaniya analizləri**: Loyallıq proqramı, endirim, kampaniya sonrası davranış dəyişikliklərini vizuallaşdır — Before/After, Cohort analiz kimi.
* **Customer Health Score**: Müxtəlif ölçüləri (alış sayı, son alış, xərcləmə, geri dönüş tezliyi, dəstək müraciətləri və s.) skorlaşdır + Composite skor → dashboard.

---

## 🎯 Niyə bu metriklər vacibdir?

* Hər müştəri eyni deyil — bəziləri “daha dəyərlidir”, bəziləri “risq altındadır”. Yalnız ümumi Retention / Churn ilə yetinmək bəzən kifayət etmir.
* Büdcəni, marketinq / kampaniya xərclərini daha düzgün bölmək — məsələn, yüksək CLV olan müştərilərə yönəlmək; risqli olanları “win-back” kampaniyası ilə geri qaytarmaq.
* Müştəri seqmentlərinə görə fərdi strategiyalar qurmaq — loyalty proqramları, endirimlər, upsell/cross-sell fürsətləri.
* Uzun müddətli biznes strategiyası qurmaq — “buyers” deyil, “long-term partners / loyal customers” yaratmaq.

---

Əgər istəyirsənsə, **Power BI + DAX** nümunələri ilə birlikdə **tam bir dashboard strukturu (template)** yarada bilərəm — yəni “Loyalty & CLV Dashboard”. Belə edək?

[1]: https://en.wikipedia.org/wiki/Customer_lifetime_value?utm_source=chatgpt.com "Customer lifetime value"
[2]: https://www.loyoly.io/blog/kpis-to-measure-customer-loyalty?utm_source=chatgpt.com "Measure customer loyalty : 11 KPIs and how to calculate them"
[3]: https://webkodlama.net/marketing-attribution-customer-lifetime-value-clv-measurement-2025-rehberi.html?utm_source=chatgpt.com "Pazarlama Performans Ölçümü ve Müşteri Yaşam Boyu Değeri: 2025 Analytics Rehberi | Hatay Web Kodlama"
[4]: https://medium.com/machine-learning-t%C3%BCrkiye/crm-analizi-rfm-analizi-ve-cltv-m%C3%BC%C5%9Fteri-ya%C5%9Fam-boyu-de%C4%9Feri-36e5c3a232b1?utm_source=chatgpt.com "CRM Analizi (RFM Analizi ve CLTV (Müşteri Yaşam Boyu Değeri)) | by Şevval Yurtekin | Machine Learning Turkiye | Medium"
[5]: https://rapitek.com/blog/musteri-yasam-dongusu-yonetimi-nedir?utm_source=chatgpt.com "Müşteri Yaşam Döngüsü Yönetimi (CLM) Nedir? 2025 Kapsamlı Rehber | Rapitek CRM Blog"


Hazırdır!
Sənin üçün **AzeriMed şirkətinə uyğun 1000 sətrlik satış datası** yaratdım.

📥 **Faylı buradan yüklə:**
👉 [AzeriMed_Sales_Data_1000.csv](sandbox:/mnt/data/AzeriMed_Sales_Data_1000.csv)

---

# 📌 İndi bu datanı Power BI-yə necə yükləyib analitika quracaqsan?

Aşağıda **addım-addım** yazıram.

---

# ✅ **1. Addım — Power BI-yə data import et**

1. Power BI Desktop aç
2. **Home → Get Data → Text/CSV** seç
3. Yüklədiyin `AzeriMed_Sales_Data_1000.csv` faylını aç
4. **Load** bas

---

# ✅ **2. Addım — Calendar (Date Table) yarat**

Bu analitika üçün vacibdir.

**Modeling → New Table**:

```DAX
Calendar = CALENDAR(MIN(Sales[OrderDate]), MAX(Sales[OrderDate]))
```

Sonra **Model View**-də əlaqə yarat:

* Calendar[Date] → Sales[OrderDate] (One-to-Many)

---

# ✅ **3. Addım — Əsas metriklər üçün Measures yarat**

### 🔵 Total Sales

```DAX
Total Sales = SUM(Sales[Amount])
```

### 🔵 Total Customers

```DAX
Total Customers = DISTINCTCOUNT(Sales[CustomerID])
```

### 🔵 Repeat Customers (minimum 2 alış edən)

```DAX
Repeat Customers =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    FILTER(
        VALUES(Sales[CustomerID]),
        CALCULATE(COUNTROWS(Sales)) > 1
    )
)
```

### 🔵 Repeat Rate

```DAX
Repeat Rate = DIVIDE([Repeat Customers], [Total Customers])
```

---

# 🔴 **4. Addım — Churn Rate (aylıq itirilən müştərilər)**

### Active Customers (Current Month)

```DAX
Active Customers =
DISTINCTCOUNT(Sales[CustomerID])
```

### Active Customers (Previous Month)

```DAX
Active Customers Previous Month =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    DATEADD(Calendar[Date], -1, MONTH)
)
```

### Lost Customers

```DAX
Lost Customers =
[Active Customers Previous Month] - [Active Customers]
```

### Churn Rate

```DAX
Churn Rate =
DIVIDE([Lost Customers], [Active Customers Previous Month])
```

---

# 🟢 **5. Addım — Retention Rate (geri qayıdanlar)**

Returning Customers:

```DAX
Returning Customers =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    FILTER(
        Sales,
        Sales[OrderDate] >= EDATE(MAX(Sales[OrderDate]), -1)
    )
)
```

Retention Rate:

```DAX
Retention Rate =
DIVIDE([Returning Customers], [Active Customers Previous Month])
```

---

# 🟣 **6. Addım — RFM Analizi (Recency, Frequency, Monetary)**

### Recency (son alışdan neçə gün keçib?)

```DAX
Recency =
DATEDIFF(
    CALCULATE(MAX(Sales[OrderDate]), ALLEXCEPT(Sales, Sales[CustomerID])),
    MAX(Calendar[Date]),
    DAY
)
```

### Frequency (alış sayı)

```DAX
Frequency =
CALCULATE(
    COUNT(Sales[OrderID]),
    ALLEXCEPT(Sales, Sales[CustomerID])
)
```

### Monetary (xərclənən məbləğ)

```DAX
Monetary =
CALCULATE(
    SUM(Sales[Amount]),
    ALLEXCEPT(Sales, Sales[CustomerID])
)
```

---

# 📊 **7. Addım — Dashboard vizualları**

### Kartlar (Card):

* Total Sales
* Total Customers
* Retention Rate
* Churn Rate
* Repeat Rate

### Qrafiklər:

* **Line Chart:** Aylıq satış trendi
* **Bar Chart:** Region üzrə satış
* **Pie Chart:** Product paylaşımı
* **Table:** RFM dəyərləri (Recency, Frequency, Monetary)

### Cohort Analysis (İstəsən əlavə edərəm)

---

# 🎁 İstəsən sənin üçün **hazır Power BI dashboard (.pbix)** də yarada bilərəm

Sadəcə de: **"dashboard hazırla"** — və mən yükləyə biləcəyin tam dashboard hazırlayım.




# 🔹 **1. Data yüklə**

1. Power BI Desktop aç
2. **Home → Get Data → Text/CSV** → `AzeriMed_Sales_Data_1000.csv` faylını seç
3. **Load** bas

---

# 🔹 **2. Əlavə sütunlar (Rule-based flags) yarat**

### **2.1 High Amount Flag**

* Modeling → **New Column**

```DAX
HighAmountFlag = IF(Sales[Amount] > 450, 1, 0)
```

* Məna: məbləği 450+ olan əməliyyat → 1, digərləri → 0

---

### **2.2 Fast Repeat Purchase (7 gün içində 3+ alış)**

* Power BI-də bir az mürəkkəbdir, amma sadə yanaşma:
* Əlavə sütun:

```DAX
OrdersLast7Days =
VAR CurrentDate = Sales[OrderDate]
VAR CustID = Sales[CustomerID]
RETURN
CALCULATE(
    COUNTROWS(Sales),
    FILTER(
        Sales,
        Sales[CustomerID] = CustID &&
        Sales[OrderDate] <= CurrentDate &&
        Sales[OrderDate] > CurrentDate - 7
    )
)
```

* Sonra FastRepeatFlag:

```DAX
FastRepeatFlag = IF(Sales[OrdersLast7Days] >= 3, 1, 0)
```

---

### **2.3 Rare Product / Region Flag**

* Rare Product:

```DAX
RareProductFlag = IF(Sales[Product] IN {"Antiviral","Syringe"}, 1, 0)
```

* Rare Region:

```DAX
RareRegionFlag = IF(Sales[Region] IN {"Nakhchivan","Shirvan"}, 1, 0)
```

---

# 🔹 **3. Fraud Score**

* Yeni Measure yarat:

```DAX
FraudScore = Sales[HighAmountFlag] + Sales[FastRepeatFlag] + Sales[RareProductFlag] + Sales[RareRegionFlag]
```

* Məna: hər flag 1 bal, toplam bal 0–4 arası

---

# 🔹 **4. Suspicious Transaction Flag**

* Yeni Column:

```DAX
SuspiciousFlag = IF(Sales[FraudScore] >= 2, 1, 0)
```

* Məna: FraudScore ≥ 2 → suspicious transaction

---

# 🔹 **5. Dashboard vizualları**

1. **Card:**

   * Total Suspicious Transactions = SUM(Sales[SuspiciousFlag])
2. **Table:**

   * CustomerID, OrderID, Amount, FraudScore, SuspiciousFlag
3. **Bar Chart:**

   * Region / Product üzrə suspicious transaction sayı
4. **Line Chart:**

   * Aylıq suspicious transaction trendi

---

# 🔹 **6. Anomaly detection vizual**

* Power BI-nin **Python / R script vizual** xüsusiyyətindən istifadə edə bilərsən:

  * Amount üzrə Z-score və ya Isolation Forest tətbiq edib, anomalies vizuallaşdırmaq
  * SuspiciousFlag ilə birləşdirərək comparison etmək

---

💡 **Qeyd:** Bu yanaşma **rule-based, statik fraud detection** üçündür. Real-time və advanced ML modellər üçün əlavə sütunlar (IP, country, payment method, card number, login time) lazım olacaq.

---

İstəsən, mən sənə **hazır Power BI .pbix faylı** hazırlayıb **yükləyə biləcəyin dashboard** da verə bilərəm, bütün flag-lar və FraudScore daxildir.

Bunu edim?


# Proses buradan bashlayir

# Excel

Əla! İndi **eyni loyallıq və fraud analizi addımlarını Excel üçün** tam şəkildə yazıram. Bu izahla sən həm **Excel-də data təmizləmə**, həm də **loyallıq + fraud analitikası** üçün hazır model qura biləcəksən.

---

# 📘 **AzeriMed üçün Excel-də Loyallıq və Fraud Analizi (Addım-Addım)**

Aşağıdakı addımlar **1000 sətrlik satış datasına** tam uyğundur.

---

# ✅ **1. Addım — Excel-ə datanı yüklə**

1. Excel-i aç
2. **Data → From Text/CSV**
3. `AzeriMed_Sales_Data_1000.csv` faylını seç
4. **Load** bas

Data gələcək:
**OrderID, CustomerID, Product, Amount, Region, OrderDate** və s.

---

# ✅ **2. Addım — Data hazırlığı (Power Query ilə)**

Excel 2016+ istifadə edirsə:

1. Data → **From Table/Range**
2. **Power Query** açılacaq
3. "OrderDate" sütununu **Date** formatına çevir
4. "Amount" sütununu **Decimal Number** et
5. Close & Load

---

# ✅ **3. Addım — Pivot Table yarat (əsas analitik baza)**

1. Insert → **PivotTable**
2. Data source → satış datası
3. New Worksheet seç

---

# 🟦 **4. Addım — Loyallıq Analizi (Excel)**

Burada aşağıdakı göstəriciləri çıxaracağıq:

### ✔ Total Sales

Pivot:

* Values → Amount (Sum)

### ✔ Total Customers

Pivot:

* Values → CustomerID (Distinct Count)
  👉 Distinct Count görsənmirsə:
  **Add this data to the Data Model** seç.

### ✔ Repeat Customers (2+ alış edən)

Bunun üçün yeni köməkçi sütun əlavə edirik:

### **Yeni sütun: PurchaseCount**

Excel-də boş sütunda yaz:

```excel
=COUNTIF(CustomerID_sutunu; CustomerID)
```

Məsələn:

```excel
=COUNTIF($B$2:$B$1001; B2)
```

Sonra Pivot Table-də:

* Filters → PurchaseCount
* Filter: **>= 2**
* Values → CustomerID (Distinct Count)

Bu sənə **Repeat Customer Count** verəcək.

### ✔ Repeat Rate

Excel-də ayrıca hüceyrədə yaz:

```excel
=Repeat_Customers / Total_Customers
```

Format → % seç.

---

# 🟥 **5. Addım — Churn Rate (aylıq itirilən müştərilər)**

### 1) Yeni sütun: **Year-Month**

Excel-də:

```excel
=TEXT(OrderDate,"yyyy-mm")
```

### 2) Pivot Table:

* Rows → Year-Month
* Values → CustomerID (Distinct Count)

Bu sənə aylıq aktiv müştəri sayını verir.

### 3) Aylıq Churn hesablamaq üçün:

**ActiveCustomers_prev**
Bir hüceyrədə:

```excel
=OFFSET(aktiv_musteri_hucresi; -1; 0)
```

**Lost Customers**

```excel
=ActiveCustomers_prev - ActiveCustomers
```

**Churn Rate**

```excel
=LostCustomers / ActiveCustomers_prev
```

---

# 🟩 **6. Addım — Retention Rate**

Retention = geri qayıdan müştərilər / əvvəlki ayın aktiv müştəriləri

Yeni sütun əlavə et:

### **Returning Flag**

Müştərinin əvvəlki ay alış edib-etməməsini tapırıq:

```excel
=IF(
   COUNTIFS(
      [CustomerID], [@CustomerID],
      [OrderDate], "<" & EOMONTH([@OrderDate], -1) + 1

```

Pivot Table-də:

* Values → Returning Flag (Sum)
* Values → Distinct Count of CustomerID


**ActiveCustomers_prev**
Bir hüceyrədə:

```excel
=OFFSET(aktiv_musteri_hucresi; -1; 0)
```


Retention Rate:

```excel
=Sum Returning Flag/ ActiveCustomers_prev
```

---

# 🟣 **7. Addım — RFM Analizi Excel-də**

RFM üçün 3 sütun əlavə edirik:

---

### ✔ **Recency**

Son alışdan neçə gün keçib?

```excel
=MAX(OrderDate_range) - MAXIFS(OrderDate_range; CustomerID_range; B2)
```

---

### ✔ **Frequency**

Müştərinin neçə alış etdiyi

```excel
=COUNTIF(CustomerID_range; B2)
```

---

### ✔ **Monetary**

Müştərinin ümumi xərci

```excel
=SUMIF(CustomerID_range; B2; Amount_range)
```

Sonra bütün bu sütunları PivotTable ilə analiz edə bilərsən.

---

# 📊 **8. Addım — Excel Dashboard (vizuallar)**

Dashboard üçün aşağıdakı vizuallar qur:

### 📌 Kartlar (Shape + Linked Cell):

* Total Sales
* Total Customers
* Repeat Rate
* Retention Rate
* Churn Rate

### 📌 Diagramlar:

* **Line Chart:** Aylıq Satış Trend
* **Column Chart:** Region üzrə satış
* **Pie Chart:** Məhsul payı
* **Bar Chart:** RFM Monetary sıralaması

### 📌 KPI göstəriciləri:

* Repeat Customer Trend
* Churn vs Retention Comparison

---

# 🎁 İstəsən Sənin üçün **hazır Excel Dashboard (.xlsx)** də yarada bilərəm

Sadəcə de:

👉 **"Excel dashboard hazırla"**

və mən sənə **hazır interaktiv Excel dashboard** hazırlayıb yükləmək üçün verim.


# Power BI

Hazırdır!
Sənin üçün **AzeriMed şirkətinə uyğun 1000 sətrlik satış datası** yaratdım.

📥 **Faylı buradan yüklə:**
👉 [AzeriMed_Sales_Data_1000.csv](sandbox:/mnt/data/AzeriMed_Sales_Data_1000.csv)

---

# 📌 İndi bu datanı Power BI-yə necə yükləyib analitika quracaqsan?

Aşağıda **addım-addım** yazıram.

---

# ✅ **1. Addım — Power BI-yə data import et**

1. Power BI Desktop aç
2. **Home → Get Data → Text/CSV** seç
3. Yüklədiyin `AzeriMed_Sales_Data_1000.csv` faylını aç
4. **Load** bas

---

# ✅ **2. Addım — Calendar (Date Table) yarat**

Bu analitika üçün vacibdir.

**Modeling → New Table**:

```DAX
Calendar = CALENDAR(MIN(Sales[OrderDate]), MAX(Sales[OrderDate]))
```

Sonra **Model View**-də əlaqə yarat:

* Calendar[Date] → Sales[OrderDate] (One-to-Many)

---

# ✅ **3. Addım — Əsas metriklər üçün Measures yarat**

### 🔵 Total Sales

```DAX
Total Sales = SUM(Sales[Amount])
```

### 🔵 Total Customers

```DAX
Total Customers = DISTINCTCOUNT(Sales[CustomerID])
```

### 🔵 Repeat Customers (minimum 2 alış edən)

```DAX
Repeat Customers =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    FILTER(
        VALUES(Sales[CustomerID]),
        CALCULATE(COUNTROWS(Sales)) > 1
    )
)
```

### 🔵 Repeat Rate

```DAX
Repeat Rate = DIVIDE([Repeat Customers], [Total Customers])
```

---

# 🔴 **4. Addım — Churn Rate (aylıq itirilən müştərilər)**

### Active Customers (Current Month)

```DAX
Active Customers =
DISTINCTCOUNT(Sales[CustomerID])
```

### Active Customers (Previous Month)

```DAX
Active Customers Previous Month =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    DATEADD(Calendar[Date], -1, MONTH)
)
```

### Lost Customers

```DAX
Lost Customers =
[Active Customers Previous Month] - [Active Customers]
```

### Churn Rate

```DAX
Churn Rate =
DIVIDE([Lost Customers], [Active Customers Previous Month])
```

---

# 🟢 **5. Addım — Retention Rate (geri qayıdanlar)**

### bu düz işləmir
Returning Customers:

```DAX
Returning Customers =
CALCULATE(
    DISTINCTCOUNT(Sales[CustomerID]),
    FILTER(
        Sales,
        Sales[OrderDate] >= EDATE(MAX(Sales[OrderDate]), -1)
    )
)
```

### Əsas budur
Returning Customer Flag

```DAX
Returning Customer Flag = 
VAR CurrCustomer = AzeriMed_Sales_Data_1000_origin[CustomerID]
VAR CurrMonthStart =
    DATE(
        YEAR(AzeriMed_Sales_Data_1000_origin[OrderDate]),
        MONTH(AzeriMed_Sales_Data_1000_origin[OrderDate]),
        1
    )
RETURN
IF (
    CALCULATE (
        COUNTROWS ( AzeriMed_Sales_Data_1000_origin ),
        FILTER (
            AzeriMed_Sales_Data_1000_origin,
            AzeriMed_Sales_Data_1000_origin[CustomerID] = CurrCustomer
                && AzeriMed_Sales_Data_1000_origin[OrderDate] < CurrMonthStart
        )
    ) > 0,
    1,
    0
)
```

Returning Customers Total

```DAX
Returning Customers Total = 
SUM(AzeriMed_Sales_Data_1000_origin[Returning Customer Flag])
```

Retention Rate:

```DAX
Retention Rate = 
DIVIDE([Returning Customers Total], [Active Customers Previous Month])
```

---

# 🟣 **6. Addım — RFM Analizi (Recency, Frequency, Monetary)**

### Recency (son alışdan neçə gün keçib?)

```DAX
Recency =
DATEDIFF(
    CALCULATE(MAX(Sales[OrderDate]), ALLEXCEPT(Sales, Sales[CustomerID])),
    MAX(Calendar[Date]),
    DAY
)
```

### Frequency (alış sayı)

```DAX
Frequency =
CALCULATE(
    COUNT(Sales[OrderID]),
    ALLEXCEPT(Sales, Sales[CustomerID])
)
```

### Monetary (xərclənən məbləğ)

```DAX
Monetary =
CALCULATE(
    SUM(Sales[Amount]),
    ALLEXCEPT(Sales, Sales[CustomerID])
)
```

---

# 📊 **7. Addım — Dashboard vizualları**

### Kartlar (Card):

* Total Sales
* Total Customers
* Retention Rate
* Churn Rate
* Repeat Rate

### Qrafiklər:

* **Line Chart:** Aylıq satış trendi
* **Bar Chart:** Region üzrə satış
* **Pie Chart:** Product paylaşımı
* **Table:** RFM dəyərləri (Recency, Frequency, Monetary)


### 📌 KPI göstəriciləri:

* Repeat Customer Trend
* Churn vs Retention Comparison


### Cohort Analysis (İstəsən əlavə edərəm)

---

# 🎁 İstəsən sənin üçün **hazır Power BI dashboard (.pbix)** də yarada bilərəm

Sadəcə de: **"dashboard hazırla"** — və mən yükləyə biləcəyin tam dashboard hazırlayım.


# Tableau

Əladır 👍
İndi **eyni dataseti Tableau üçün** tam uyğun şəkildə, **addım-addım** yazıram. Power BI → Tableau məntiqinə çevirirəm.

---

# 📊 **AzeriMed – Tableau üçün Satış Analitikası Bələdçisi**

---

## ✅ **1. Addım — Data-nı Tableau-ya import et**

1. **Tableau Desktop** aç
2. **Connect → Text File** seç
3. `AzeriMed_Sales_Data_1000.csv` faylını seç
4. **Sheet 1**-ə keç

> Tableau avtomatik `OrderDate`-i **Date** tipinə çevirəcək

---

## ✅ **2. Addım — Date (Calendar) məntiqi**

Tableau-da ayrıca Calendar table yaratmaq **məcburi deyil**.

Sadəcə:

* `OrderDate` üzərində **YEAR / MONTH / DAY** istifadə edirik
* Lazım olsa: `DATETRUNC()` funksiyası ilə ay, il səviyyəsi

Məsələn:

```tableau
DATETRUNC('month', [OrderDate])
```

---

## ✅ **3. Addım — Əsas KPI-lar (Calculated Fields)**

### 🔵 Total Sales

```tableau
SUM([Amount])
```

---

### 🔵 Total Customers

```tableau
COUNTD([CustomerID])
```

---

### 🔵 Repeat Customers (2+ alış edənlər)

```tableau
IF { FIXED [CustomerID] : COUNT([OrderID]) } > 1 THEN 1 END
```

➡ Sonra:

```tableau
SUM([Repeat Customers])
```

---

### 🔵 Repeat Rate

```tableau
SUM([Repeat Customers]) / COUNTD([CustomerID])
```

---

# 🔴 **4. Addım — Churn Rate (aylıq itirilən müştərilər)**

### 🔵 Active Customers (Current Month)

```tableau
COUNTD([CustomerID])
```

---

### 🔵 Active Customers (Previous Month)

```tableau
COUNTD(
IF DATETRUNC('month',[OrderDate]) =
DATEADD('month',-1,DATETRUNC('month',TODAY()))
THEN [CustomerID] END
)
```

---

### 🔵 Lost Customers

```tableau
[Active Customers Previous Month] - [Active Customers]
```

---

### 🔵 Churn Rate

```tableau
[Lost Customers] / [Active Customers Previous Month]
```

---

# 🟢 **5. Addım — Retention Rate (geri qayıdanlar)**

### 🔵 Returning Customer Flag

(Bu Power BI-dəki düzgün məntiqin Tableau versiyasıdır)

```tableau
IF
{ FIXED [CustomerID] :
MIN([OrderDate])
}
<
DATETRUNC('month',[OrderDate])
THEN 1 END
```

---

### 🔵 Returning Customers Total

```tableau
SUM([Returning Customer Flag])
```

---

### 🔵 Retention Rate

```tableau
SUM([Returning Customer Flag]) / [Active Customers Previous Month]
```

---

# 🟣 **6. Addım — RFM Analizi (Recency, Frequency, Monetary)**

### 🔵 Recency (son alışdan neçə gün keçib)

```tableau
DATEDIFF(
'day',
{ FIXED [CustomerID] : MAX([OrderDate]) },
TODAY()
)
```

---

### 🔵 Frequency (alış sayı)

```tableau
{ FIXED [CustomerID] : COUNT([OrderID]) }
```

---

### 🔵 Monetary (ümumi məbləğ)

```tableau
{ FIXED [CustomerID] : SUM([Amount]) }
```

---

## 📊 **7. Addım — Dashboard Vizual Tövsiyələri**

### 🟦 KPI Cards:

* Total Sales
* Total Customers
* Repeat Rate
* Retention Rate
* Churn Rate

---

### 📈 Qrafiklər:

* **Line Chart:** `MONTH(OrderDate)` → `SUM(Amount)`
* **Bar Chart:** `Region` → `SUM(Amount)`
* **Pie Chart:** `Product` → `SUM(Amount)`
* **Table:** `CustomerID | Recency | Frequency | Monetary`

---

### 📌 Advanced:

* Repeat vs New Customers (stacked bar)
* Churn vs Retention (line / area)
* **Cohort Analysis** (istəsən ayrıca yazaram)

---

### 📌 KPI göstəriciləri:

* Repeat Customer Trend
* Churn vs Retention Comparison


## 🎁 İstəsən nə edə bilərəm?

✔ Hazır **Tableau Dashboard dizaynı**
✔ **Cohort Analysis**
✔ **RFM segmentləri (VIP, Loyal, At Risk)**
✔ **Interview üçün Tableau case**

Sadəcə yaz:
👉 **“Tableau dashboard hazırla”**
və ya
👉 **“RFM segmentləşmə et”**

İstədiyin səviyyəyə görə davam edək 💪
